Laplace mixed boundary values
==============================

**keys**: homogeneous Laplace bvp, Calderon projector, Dirichlet data, Neumann data 

In [2]:
import sys
sys.path.append("../build/")
from ngsolve import *
from netgen.occ import *
from ngsolve.krylovspace import CG, GMRes
from ngsolve.webgui import Draw
from libbem import *

Loading ngbem library


Consider the following interior boundary value problem with mixed boundary conditions: 

$$ \begin{array}{r rcl r} & \Delta u &=& 0  &\mathrm{in}\; \Omega\,,\\ \textnormal{Dirichlet condition} & \gamma_0 u &=& u_0 & \mathrm{on}\; \Gamma_0\,,\\ \textnormal{Neumann condition} & \gamma_1 u &=&  u_1 & \mathrm{on}\; \Gamma_1\,. \end{array} $$ 

Let for instance $\Omega$ be a unit ball and $\Gamma_0$ the lower half sphere and $\Gamma_1$ the upper half sphere.

In [4]:
topsphere = Sphere((0,0,0), 1) * Box((-1,-1,0),(1,1,1))
botsphere = Sphere((0,0,0), 1) - Box((-1,-1,0),(1,1,1))
topsphere.faces.name = "neumann"
botsphere.faces.name = "dirichlet"
shape = Fuse( [topsphere,botsphere] )

order = 3
mesh = Mesh(OCCGeometry(shape).GenerateMesh(maxh=0.25)).Curve(order)
Draw (mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

The following representation formula for the solution $u$ holds:

$$ x \in \Omega: \quad u(x) =  \int_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{1}{\| x-y\|} } \, \gamma_1 u (y)\, \mathrm{d}\sigma_y - \int_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{\langle n(y) , x-y\rangle }{\| x-y\|^3} } \, \gamma_0 u (y)\, \mathrm{d}\sigma_y\,, $$ 

and the unique traces of $u$ are related by the Calderon projector

  
$$  \left( \begin{array}{c} \gamma_0 u \\ \gamma_1 u \end{array}\right) =  \left( \begin{array}{cc} -V & \frac12 + K \\ \frac12 - K^\intercal & -D \end{array} \right)  \left( \begin{array}{c} \gamma_1 u \\ \gamma_0 u \end{array}\right) $$

In [20]:
fesL2 = SurfaceL2(mesh, order=order-1, dirichlet="neumann")
u,v = fesL2.TnT()
fesH1 = H1(mesh, order=order, dirichlet="dirichlet", definedon=mesh.Boundaries(".*"))
u1,v1 = fesH1.TnT()
print ("ndofL2 = ", fesL2.ndof, "ndofH1 = ", fesH1.ndof)

ndofL2 =  2736 ndofH1 =  2128


In [21]:
uexa = CF(x)
ud = GridFunction(fesH1)
ud.Interpolate(uexa, definedon=mesh.Boundaries("dirichlet"))

n = specialcf.normal(3)
gradn_uexa = CF((uexa.Diff(x), uexa.Diff(y), uexa.Diff(z))) * n
un = GridFunction(fesL2)
un.Interpolate(gradn_uexa, definedon=mesh.Boundaries("neumann"))
Draw(ud, mesh, draw_vol=False)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [22]:
eps = 1e-6
intorder = 2 * order + 6
with TaskManager():
    V = SingleLayerPotentialOperator(fesL2, intorder=intorder, eps=eps)
    K = DoubleLayerPotentialOperator(fesH1, fesL2, intorder=intorder, eps=eps)
    W = HypersingularOperator(fesH1, intorder=intorder, eps=eps)
    M = BilinearForm(u1.Trace() * v.Trace() * ds(bonus_intorder=3)).Assemble()
    fd = ((0.5 * M.mat + K.mat) * ud.vec - V.mat * un.vec).Evaluate()
    fn = ((0.5 * M.mat.T - K.mat.T) * un.vec - W.mat * ud.vec).Evaluate()
    pred = BilinearForm(u.Trace() * v.Trace() * ds(bonus_intorder=3), check_unused=False).Assemble()
    pren = BilinearForm(u1.Trace() * v1.Trace() * ds(bonus_intorder=3), check_unused=False).Assemble()

In [26]:
lhs = BlockMatrix([[V.mat, - K.mat], [K.mat.T, W.mat]])
rhs = BlockVector([fd, fn])
pre = BlockMatrix ([[pred.mat.Inverse(freedofs=fesL2.FreeDofs()), None], [None, pren.mat.Inverse(freedofs=fesH1.FreeDofs())]])

sol = GMRes(A=lhs, b=rhs, pre=pre, maxsteps=300)

GMRes iteration 1, residual = 109.56977428387229     
GMRes iteration 2, residual = 53.5780889864675     
GMRes iteration 3, residual = 20.02971529993262     
GMRes iteration 4, residual = 7.726387210439952     
GMRes iteration 5, residual = 5.904571084454502     
GMRes iteration 6, residual = 5.225276193491822     
GMRes iteration 7, residual = 4.695885280589744     
GMRes iteration 8, residual = 3.845742237212525     
GMRes iteration 9, residual = 2.934693441298939     
GMRes iteration 10, residual = 2.1473652961829655     
GMRes iteration 11, residual = 1.798482243956206     
GMRes iteration 12, residual = 1.609659909950845     
GMRes iteration 13, residual = 1.4221319492240578     
GMRes iteration 14, residual = 1.283385497448402     
GMRes iteration 15, residual = 1.1815881666909356     
GMRes iteration 16, residual = 1.0835527239355531     
GMRes iteration 17, residual = 0.9660125018594617     
GMRes iteration 18, residual = 0.8510309673852624     
GMRes iteration 19, residual = 

In [27]:
gfu = GridFunction(fesL2)
gfu.vec[:] = sol[0]
#gfu.Interpolate(gradn_uexa, definedon=mesh.Boundaries("neumann")) 
Draw(gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [28]:
print ("L2 error of surface gradients =", sqrt(Integrate((gradn_uexa - gfu)**2, mesh.Boundaries("dirichlet"), BND)))

L2 error of surface gradients = 0.0004377141341219706
